# 第十章_评价分类结果

## 10-1 准确度的陷阱和混淆矩阵

对于回归问题可使用 `MAE`, `MSE`, `RMSE` 以及最好的评价回归的指标`R Squard`, 但是对于分类算法的评价使用**分类准确度**这一指标

分类准确度的问题：  
搭建一个癌症预测系统，输入体检信息，可以判断是否有癌症  
预测准确度：99.9%  
如果癌症产生的概率只有0.01%  
我们系统预测所有人都是健康的，可达到99.99%的准确度  

对于极度偏斜 (Skewed Data）的数据只使用分类准确度 是远远不够的，需要使用 混淆矩阵 做进一步的分析  

#### 混淆矩阵 `Confusion Matrix`

![](picture/10-01-01.png)

![](picture/10-01-02.png)
9978: 表示预测没有患病的人数 `TN`  
12: 表示患者没有患癌症, 但预测该患者患有癌症, 预测失败的人数 `FP`   
2: 表示患者患有癌症, 但是预测却没有癌症, 预测失败的人数 `FN`  
8: 表示患者没有癌症, 预测结果也没有患有癌症, 预测正确的人数 `TP` 

## 10-2 精准率和召回率

![](picture/10-02-01.png)

![](picture/10-02-02.png)

精准率: **做了100次分类为1的预测**, 其中有多少次是正确的. (比如: **预测有100个人有癌症**, 在这些预测中有多少是准确的)  
召回率: **实际上有100个我们关注的时间发生**, 其中有多少个是准确的.(比如: **实际上有100个癌症病人**, 预测算法从中能正确的挑出多少个癌症病人)

![](picture/10-02-03.png)

![](picture/10-02-04.png)

## 10-3 实现混淆矩阵-精准率和召回率

In [2]:
import numpy as np
from sklearn import datasets

In [7]:
digits = datasets.load_digits()
X = digits.data
y = digits.target.copy()

"""将数字为9的标记值为1, 其余的全部设置为0,
将数据转换为具有极度偏差的数据
"""
y[digits.target == 9] = 1
y[digits.target != 9] = 0

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [18]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver="liblinear")
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.9755555555555555

In [20]:
y_log_predict = log_reg.predict(X_test)

In [21]:
def TN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 0))

TN(y_test, y_log_predict)

403

In [22]:
def FP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 1))

FP(y_test, y_log_predict)

2

In [23]:
def FN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 0))

FN(y_test, y_log_predict)

9

In [24]:
def TP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 1))

TP(y_test, y_log_predict)

36

In [25]:
def confusion_matrix(y_true, y_predict):
    return np.array([
        [TN(y_true, y_predict), FP(y_true, y_predict)],
        [FN(y_true, y_predict), TP(y_true, y_predict)]
    ])

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [26]:
def precision_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fp = FP(y_true, y_predict)
    try:
        return tp / (tp + fp)
    except:
        return 0.0
    
precision_score(y_test, y_log_predict)

0.9473684210526315

In [27]:
def recall_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fn = FN(y_true, y_predict)
    try:
        return tp / (tp + fn)
    except:
        return 0.0
    
recall_score(y_test, y_log_predict)

0.8

## scikit-learn中的混淆矩阵，精准率和召回率

In [28]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [29]:
from sklearn.metrics import precision_score

precision_score(y_test, y_log_predict)

0.9473684210526315

In [30]:
from sklearn.metrics import recall_score

recall_score(y_test, y_log_predict)

0.8

## 10-4 F1 Score

股票预测, 精准率重要些  
病人预测, 召回率重要些

![](picture/10-04-01.png)

![](picture/10-04-02.png)

In [31]:
import numpy as np

In [32]:
def f1_score(precision, recall):
    try:
        return 2 * precision * recall / (precision + recall)
    except:
        return 0.0

In [33]:
precision = 0.5
recall = 0.5
f1_score(precision, recall)

0.5

In [34]:
precision = 0.1
recall = 0.9
f1_score(precision, recall)

0.18000000000000002

In [35]:
precision = 0.0
recall = 1.0
f1_score(precision, recall)

0.0

In [36]:
from sklearn import datasets

digits = datasets.load_digits()
X = digits.data
y = digits.target.copy()

y[digits.target==9] = 1
y[digits.target!=9] = 0

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [46]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver="liblinear")
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.9755555555555555

In [47]:
y_predict = log_reg.predict(X_test)

In [48]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_predict)

array([[403,   2],
       [  9,  36]])

In [49]:
from sklearn.metrics import precision_score

precision_score(y_test, y_predict)

0.9473684210526315

In [50]:
from sklearn.metrics import recall_score

recall_score(y_test, y_predict)

0.8

In [51]:
from sklearn.metrics import f1_score

f1_score(y_test, y_predict)

0.8674698795180723

## 10-5 Precision-Recall的平衡

In [52]:
import numpy as np
import matplotlib.pyplot as plt

In [53]:
from sklearn import datasets

digits = datasets.load_digits()
X = digits.data
y = digits.target.copy()

y[digits.target==9] = 1
y[digits.target!=9] = 0

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [55]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_predict = log_reg.predict(X_test)

/home/ubuntu/Env/py3tf/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [56]:
from sklearn.metrics import f1_score

f1_score(y_test, y_predict)

0.8674698795180723

In [57]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_predict)

array([[403,   2],
       [  9,  36]])

In [58]:
from sklearn.metrics import precision_score

precision_score(y_test, y_predict)

0.9473684210526315

In [59]:
from sklearn.metrics import recall_score

recall_score(y_test, y_predict)

0.8

In [60]:
log_reg.decision_function(X_test)[:10]

array([-22.05695252, -33.02935768, -16.21330315, -80.37911802,
       -48.25122108, -24.54001722, -44.39165048, -25.04289608,
        -0.97825371, -19.71741442])

In [61]:
log_reg.predict(X_test)[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [62]:
decision_scores = log_reg.decision_function(X_test)

In [63]:
np.min(decision_scores)

-85.68604588355328

In [64]:
np.max(decision_scores)

19.889627156334733

In [65]:
y_predict_2 = np.array(decision_scores >= 5, dtype='int')

In [66]:
confusion_matrix(y_test, y_predict_2)

array([[404,   1],
       [ 21,  24]])

In [67]:
precision_score(y_test, y_predict_2)

0.96

In [68]:
recall_score(y_test, y_predict_2)

0.5333333333333333

In [69]:
y_predict_3 = np.array(decision_scores >= -5, dtype='int')

In [70]:
confusion_matrix(y_test, y_predict_3)

array([[390,  15],
       [  5,  40]])

In [71]:
precision_score(y_test, y_predict_3)

0.7272727272727273

In [72]:
recall_score(y_test, y_predict_3)

0.8888888888888888